In [1]:
%pip install loguru==0.7.3 python-dotenv==1.0.1 PyYAML==6.0.2 torch==2.5.1 tqdm==4.67.1 typer==0.15.1 matplotlib==3.10.0 pyarrow==18.1.0 setuptools==75.1.0 protobuf==4.25.3 ultralytics==8.3.85 ray==2.43.0 albumentations==2.0.5 pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO, settings
import gc
import json
import locale
import os
import pandas as pd
import sys
import torch
import wandb
import yaml

sys.dont_write_bytecode = True
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
config_data = """
model: "yolo12n"
wandb_project: "EyeInTheSky_merged"
data: "VisDrone.yaml"
train:
  model: "yolo12n"
  project: "EyeInTheSky"
  data: "VisDrone.yaml"
  task: detect
  epochs: 1
  batch: 16
  workers: 8
  seed: 42
  plots: True
  imgsz: 640
  exist_ok: False
  save: True
  save_period: 10
  val: True
  warmup_epochs: 10
  visualize: True
  show: True
  single_cls: False # (bool) train multi-class data as single-class
  rect: False # (bool) rectangular training if mode='train' or rectangular validation if mode='val'
  cos_lr: False
  resume: False
  amp: True # (bool) Automatic Mixed Precision (AMP) training, choices=[True, False], True runs AMP check
  fraction: 1.0
  freeze: None
  cache: False
val:
  project: "EyeInTheSky"
  name: "YOLOv12-VisDrone-Validation"
  half: True
  conf: 0.25
  iou: 0.6
  split: "test"
  rect: True
  plots: True
  visualize: True
"""

In [4]:
def get_device() -> str:
    try:
        return 0 if torch.cuda.is_available() else "cpu"
    except Exception as e:
        print(f"Error setting device: {e}")

In [5]:
# Load config

# config = Config.load("../config/config.yaml")
config = yaml.safe_load(config_data)
config.update({"device" : get_device()})

In [6]:
def get_wandb_key_colab() -> str:
    try:
        from google.colab import userdata # type: ignore

        if userdata.get("WANDB_API_KEY") is not None:
            return userdata.get("WANDB_API_KEY")
        else:
            raise ValueError("No WANDB key found")
    except:
        return None

def get_wandb_env(path: Path) -> str:
    try:
        from dotenv import dotenv_values # type: ignore

        """Get W&B API key from Colab userdata or environment variable"""

        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Could not find .env file at {path}")

        print(f"Loading secrets from {path}")

        secrets = dotenv_values(path)
        print(f"Found keys: {list(secrets.keys())}")

        if "WANDB_API_KEY" not in secrets:
            raise KeyError(f"WANDB_API_KEY not found in {path}. Available keys: {list(secrets.keys())}")

        return secrets['WANDB_API_KEY']
    except:
        return None

def get_wandb_key(path: Path = "../.env") -> str:
    return get_wandb_key_colab() if get_wandb_key_colab() is not None else get_wandb_env(path)

In [7]:
wandb_api_key = get_wandb_key()
wandb.login(key=wandb_api_key, relogin=True)
wandb.init(
    project=config["wandb_project"],
    name=f"{config['model']}_{config['data']}_train",
)
settings.update({"wandb": True})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Loading secrets from ..\.env
Found keys: ['ROBOFLOW_API_KEY', 'WANDB_API_KEY']


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\franc\_netrc
wandb: Currently logged in as: francescoperagine (francescoperagine-universit-degli-studi-di-bari-aldo-moro) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
from ultralytics.data.dataset import YOLODataset
from ultralytics.models.yolo.detect import DetectionTrainer
from ultralytics.utils import colorstr, LOGGER
import numpy as np

class VisDroneDataset(YOLODataset):
    """
    Custom dataset for VisDrone that merges pedestrian (0) and people (1) classes.
    Handles class remapping at the earliest possible stage.
    """
    
    # Define the merged names as a class attribute to be accessible from the trainer
    merged_names = {
        0: 'persona',
        1: 'bicicletta',
        2: 'auto',
        3: 'furgone',
        4: 'camion',
        5: 'triciclo',
        6: 'triciclo-tendato',
        7: 'autobus',
        8: 'motociclo'
    }
    
    def __init__(self, *args, **kwargs):
        # Store original data before initialization if it exists in kwargs
        self.original_data = kwargs.get('data', {}).copy() if 'data' in kwargs else None
        
        # Adjust data names before parent initialization to make verification pass
        if self.original_data and 'names' in self.original_data:
            # Create a temporary data object with 10 classes for verification
            temp_data = self.original_data.copy()
            # Ensure we have all 10 original class names for validation
            if len(temp_data.get('names', {})) != 10:
                temp_data['names'] = {
                    0: 'pedestrian',
                    1: 'people',
                    2: 'bicycle',
                    3: 'car',
                    4: 'van',
                    5: 'truck',
                    6: 'tricycle',
                    7: 'awning-tricycle',
                    8: 'bus',
                    9: 'motor'
                }
            # Replace data in kwargs
            kwargs['data'] = temp_data
        
        # Initialize parent class with modified kwargs
        super().__init__(*args, **kwargs)
        
        # Log class mapping
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Using merged classes: {self.merged_names}")
    
    def get_labels(self):
        """
        Load and process labels with class remapping.
        """
        # Get labels from parent method
        labels = super().get_labels()
        
        # Process statistics
        people_count = 0
        shifted_count = 0
        
        # Process labels to merge classes
        for i in range(len(labels)):
            cls = labels[i]['cls']
            
            if len(cls) > 0:
                # Count 'people' instances
                people_mask = cls == 1
                people_count += np.sum(people_mask)
                
                # Merge class 1 (people) into class 0 (pedestrian -> person)
                cls[people_mask] = 0
                
                # Shift classes > 1 down by 1
                gt1_mask = cls > 1
                shifted_count += np.sum(gt1_mask)
                cls[gt1_mask] -= 1
                
                # Store modified labels
                labels[i]['cls'] = cls
        
        # Now set correct class count and names for training
        if hasattr(self, 'data'):
            # Update names and class count
            self.data['names'] = self.merged_names
            self.data['nc'] = len(self.merged_names)
        
        # Log statistics
        person_count = sum(np.sum(label['cls'] == 0) for label in labels)
        LOGGER.info(f"\n{colorstr('VisDroneDataset:')} Remapped {people_count} 'people' instances to {self.merged_names[0]}")
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Total 'persona' instances after merge: {person_count}")
        LOGGER.info(f"{colorstr('VisDroneDataset:')} Shifted {shifted_count} instances of other classes")
        
        return labels

class MergedClassDetectionTrainer(DetectionTrainer):
    """
    Custom trainer that uses VisDroneDataset for merged class training.
    """
    
    def build_dataset(self, img_path, mode="train", batch=None):
        """Build custom VisDroneDataset."""
        return VisDroneDataset(
            img_path=img_path,
            imgsz=self.args.imgsz,
            batch_size=batch or self.batch_size,
            augment=mode == "train",
            hyp=self.args,
            rect=self.args.rect if mode == "train" else True,
            cache=self.args.cache or None,
            single_cls=self.args.single_cls,
            stride=self.stride,
            pad=0.0 if mode == "train" else 0.5,
            prefix=colorstr(f"{mode}: "),
            task=self.args.task,
            classes=None,
            data=self.data,
            fraction=self.args.fraction if mode == "train" else 1.0,
        )
    
    def set_model_attributes(self):
        """Update model attributes for merged classes."""
        # First call parent method to set standard attributes
        super().set_model_attributes()
        
        # Then update model with the merged class names
        if hasattr(self.model, 'names'):
            # Use the merged names directly from the dataset class
            self.model.names = VisDroneDataset.merged_names
            self.model.nc = len(VisDroneDataset.merged_names)
            
            # Also update data dictionary
            if hasattr(self, 'data'):
                self.data['names'] = VisDroneDataset.merged_names
                self.data['nc'] = len(VisDroneDataset.merged_names)

from ultralytics.models.yolo.detect import DetectionValidator

class MergedClassDetectionValidator(DetectionValidator):
    """
    Custom validator that uses VisDroneDataset for validation/testing with merged classes.
    """
    
    def build_dataset(self, img_path, mode="val", batch=None):
        """Build custom VisDroneDataset for validation."""
        return VisDroneDataset(
            img_path=img_path,
            imgsz=self.args.imgsz,
            batch_size=batch or self.args.batch,
            augment=False,  # no augmentation during validation
            hyp=self.args,
            rect=True,  # rectangular validation for better performance
            cache=None,
            single_cls=self.args.single_cls,
            stride=self.stride,
            pad=0.5,
            prefix=colorstr(f"{mode}: "),
            task=self.args.task,
            classes=self.args.classes,
            data=self.data,
        )
    
    def set_model_attributes(self):
        """Update model attributes for merged classes if using a PyTorch model."""
        super().set_model_attributes()
        
        # Update model names if it's a PyTorch model (not for exported models)
        if hasattr(self.model, 'names') and hasattr(self.model, 'model'):
            self.model.names = VisDroneDataset.merged_names
            if hasattr(self.data, 'names'):
                self.data['names'] = VisDroneDataset.merged_names
                self.data['nc'] = len(VisDroneDataset.merged_names)

In [9]:
# Load top 5 tune results by fitness

csv_path = "../data/processed/wandb_export_2025-03-05T10_24_46.923+01_00.csv"
df = pd.read_csv(csv_path)
df = df.rename(columns={'Name': 'name'})

df['fitness'] = df['metrics/mAP50(B)'] * 0.1 + df['metrics/mAP50-95(B)'] * 0.9
df_sorted = df.sort_values(by='fitness', ascending=False).head(5)

columns_to_show = ['name', 'fitness', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)',
                  'metrics/precision(B)', 'metrics/recall(B)', 'optimizer',
                  'lr0', 'lrf', 'momentum', 'weight_decay', 'cos_lr', 'imgsz', 'box', 'cls', 'dfl']

top_5 = df_sorted[columns_to_show].reset_index(drop=True)
print("Top 5 Models by Fitness Score:")
display(top_5)

Top 5 Models by Fitness Score:


,name,fitness,metrics/mAP50(B),metrics/mAP50-95(B),metrics/precision(B),metrics/recall(B),optimizer,lr0,lrf,momentum,weight_decay,cos_lr,imgsz,box,cls,dfl
0,_tune_8290d_00015,0.184317,0.30057,0.17140,0.42157,0.30252,AdamW,0.001,0.10,0.8,0.001,True,640.0,3.872225,0.947276,4.117353
1,_tune_9e260_00006,0.183925,0.29899,0.17114,0.39727,0.30826,AdamW,0.001,0.10,0.8,0.000,True,640.0,5.120607,0.816281,4.608205
2,_tune_8290d_00008,0.183161,0.29657,0.17056,0.39720,0.30612,AdamW,0.001,0.01,0.8,0.000,True,640.0,3.427384,0.759476,3.358324
3,_tune_8290d_00012,0.182955,0.29973,0.16998,0.42051,0.30218,AdamW,0.001,0.01,0.8,0.000,True,640.0,7.015643,1.946285,4.618466
4,_tune_9e260_00015,0.182941,0.29761,0.17020,0.40803,0.30691,AdamW,0.001,0.10,0.8,0.000,True,640.0,8.909576,0.940505,0.985378


In [10]:
# Clear cache after each run

def clear_cache():
    # Clear CUDA cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Clear Python garbage collector
    gc.collect()

In [11]:
# Store results

def save_results(dir, name, results):
    os.makedirs(dir, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_path = f"{dir}/{name}_{timestamp}.json"

    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4, default=str)
    
    print(f"{name} results saved to {results_path}")

In [12]:
# model = YOLO('yolo12n.yaml')

# # Train with the custom trainer
# results = model.train(
#     **config['train'],
#     trainer=MergedClassDetectionTrainer
#     )

# test_results = model.val(
#     **config['val'],
#     validator=MergedClassValidator
#     )

# wandb.finish()

In [13]:
def start(model: YOLO, config):
    train_results = model.train(
        **config['train'],
        trainer=MergedClassDetectionTrainer
        )

    test_results = model.val(
        **config['val'],
        validator=MergedClassDetectionValidator
        )

    wandb.finish()
    return train_results, test_results

In [ ]:
model = YOLO('yolo12n.yaml')
df_train = top_5[['name', 'optimizer', 'lr0', 'lrf', 'momentum', 'weight_decay', 'cos_lr', 'box', 'cls', 'dfl']]

for idx, trial in df_train.iterrows():
    trial_config = config.copy()

    trial_config.update(trial)

    train_results, test_results = start(model, trial_config)

    save_results("../data/processed", "train", train_results)
    save_results("../data/processed", "test", test_results)

    clear_cache()

Ultralytics 8.3.85  Python-3.10.0 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i7-1270P)
engine\trainer: task=detect, mode=train, model=yolo12n, data=VisDrone.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=EyeInTheSky, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=True, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

train: Scanning C:\Users\franc\Documents\Repository\datasets\VisDrone\VisDrone2019-DET-train\labels.cache... 6471 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:00<?, ?it/s]

train: WARNING  C:\Users\franc\Documents\Repository\datasets\VisDrone\VisDrone2019-DET-train\images\0000137_02220_d_0000163.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\franc\Documents\Repository\datasets\VisDrone\VisDrone2019-DET-train\images\0000140_00118_d_0000002.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\franc\Documents\Repository\datasets\VisDrone\VisDrone2019-DET-train\images\9999945_00000_d_0000114.jpg: 1 duplicate labels removed
train: WARNING  C:\Users\franc\Documents\Repository\datasets\VisDrone\VisDrone2019-DET-train\images\9999987_00000_d_0000049.jpg: 1 duplicate labels removed

VisDroneDataset: Remapped 27059 'people' instances to persona
VisDroneDataset: Total 'persona' instances after merge: 106394
VisDroneDataset: Shifted 236807 instances of other classes


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
VisDroneDataset: Using merged classes: {0: 'persona', 1: 'bicicletta', 2: 'auto', 3: 'furgone', 4: 'camion', 5: 'triciclo', 6: 'triciclo-tendato', 7: 'autobus', 8: 'motociclo'}


val: Scanning C:\Users\franc\Documents\Repository\datasets\VisDrone\VisDrone2019-DET-val\labels.cache... 548 images, 0 backgrounds, 0 corrupt: 100%|██████████| 548/548 [00:00<?, ?it/s]

...

lr/pg0,▁
lr/pg1,▁
lr/pg2,▁
metrics/mAP50(B),▁
metrics/mAP50-95(B),▁
metrics/precision(B),▁
metrics/recall(B),▁
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▁


Ultralytics 8.3.85  Python-3.10.0 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i7-1270P)
YOLO12n summary (fused): 159 layers, 2,558,678 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\Users\franc\Documents\Repository\datasets\VisDrone\VisDrone2019-DET-test-dev\labels.cache... 1610 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1610/1610 [00:00<?, ?it/s]


VisDroneDataset: Remapped 6376 'people' instances to persona
VisDroneDataset: Total 'persona' instances after merge: 27382
VisDroneDataset: Shifted 47720 instances of other classes
VisDroneDataset: Using merged classes: {0: 'persona', 1: 'bicicletta', 2: 'auto', 3: 'furgone', 4: 'camion', 5: 'triciclo', 6: 'triciclo-tendato', 7: 'autobus', 8: 'motociclo'}



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  71%|███████▏  | 72/101 [27:04<12:10, 25.17s/it]